In [1]:
import toy_data.sine_wave as swg
import numpy as np
import bokeh.io
import bokeh.plotting as bp
import tensorflow as tf
bokeh.io.output_notebook()

d_x = np.array([int(i) for i in '0010010010010010010'])[np.newaxis, :, np.newaxis]
x_dim = 1
d_X = np.concatenate([d_x]*x_dim, axis=2)

Loading BokehJS ...

In [2]:
d_X.shape # No. of samples, Time, No. of dimensions

(1, 19, 1)

In [3]:
batch_size = 1

In [4]:
ef = lambda x: tf.expand_dims(x, -1) # expand forward
eb = lambda x: tf.expand_dims(x, 0) # expand backward

In [141]:
state_size = 10
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [batch_size, None, x_dim]) # [batch_size, num_steps]
x_in = x[:,:-1, :]
x_out = x[:, 1:, :]
cell = tf.contrib.rnn.BasicLSTMCell(state_size)
rnn_out, final_state = tf.nn.dynamic_rnn(cell, x_in, dtype=tf.float32)

W = tf.get_variable('output_embedding', (state_size, x_dim), initializer=tf.truncated_normal_initializer())
out_pred_logit = tf.reduce_sum(ef(rnn_out) * eb(eb(W)), axis=-2)
out_pred = tf.sigmoid(out_pred_logit)
loss_all = tf.nn.sigmoid_cross_entropy_with_logits(logits=out_pred_logit, labels=x_out)
loss = tf.reduce_sum(loss_all)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(loss)

gen_seq_len = tf.placeholder(tf.int32, [])
gen_seq_states = tf.transpose(tf.tile(final_state, (1, gen_seq_len, 1)), [1, 0, 2])

def update_state(_state, _):
    h = eb(_state[1, :])
    c = eb(_state[0, :])
    _pred_logit = tf.matmul(h, W)
    _pred = tf.sigmoid(_pred_logit)
    _states_tuple = tf.nn.rnn_cell.LSTMStateTuple(h, c)
    _, _next_state = cell(_pred, _states_tuple)
    return tf.concat((_next_state.h, _next_state.c), axis=0)
    
next_states = tf.scan(update_state, gen_seq_states)
next_preds = tf.map_fn(lambda s: tf.sigmoid(tf.matmul(eb(s[1, :]), W)), next_states)

In [142]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [143]:
inputs = {x: d_X}
gen_inputs = lambda n: {x: d_X, gen_seq_len: n}
for i in range(1000):
    sess.run(train_step, feed_dict=inputs)

In [144]:
print(sess.run(loss, feed_dict=inputs))
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
print('Actual value\n',sess.run(x_out, feed_dict=inputs).ravel())
print('Predictions\n', sess.run(out_pred, feed_dict=inputs).ravel())
# print('Continue1\n', sess.run(cell(out_pred[:, -1, :], final_state, scope='rnn/gru_cell'), feed_dict=inputs))
print('Continue\n', sess.run(next_preds, feed_dict=gen_inputs(100)).ravel())

4.90539e-07
Actual value
 [ 0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00]
Predictions
 [ 0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00]
Continue
 [ 0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00  0.00  1.00  0.00
  0.00  1.00  0.00  0.00]
